<a href="https://colab.research.google.com/github/JonNData/SQL/blob/master/Nguyen_Winter_2021_Data_Science_Intern_Challenge_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Question 1: Given some sample data, write a program to answer the following:
[click here to access the required data set](https://docs.google.com/spreadsheets/d/16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM/edit#gid=0)

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


In [11]:
# First I'll import the google sheet directly into colab

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [13]:
wb = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM/edit#gid=0')

data = wb.sheet1.get_all_values()
data

[['order_id',
  'shop_id',
  'user_id',
  'order_amount',
  'total_items',
  'payment_method',
  'created_at'],
 ['1', '53', '746', '224', '2', 'cash', '2017-03-13 12:36:56'],
 ['2', '92', '925', '90', '1', 'cash', '2017-03-03 17:38:52'],
 ['3', '44', '861', '144', '1', 'cash', '2017-03-14 4:23:56'],
 ['4', '18', '935', '156', '1', 'credit_card', '2017-03-26 12:43:37'],
 ['5', '18', '883', '156', '1', 'credit_card', '2017-03-01 4:35:11'],
 ['6', '58', '882', '138', '1', 'credit_card', '2017-03-14 15:25:01'],
 ['7', '87', '915', '149', '1', 'cash', '2017-03-01 21:37:57'],
 ['8', '22', '761', '292', '2', 'cash', '2017-03-08 2:05:38'],
 ['9', '64', '914', '266', '2', 'debit', '2017-03-17 20:56:50'],
 ['10', '52', '788', '146', '1', 'credit_card', '2017-03-30 21:08:26'],
 ['11', '66', '848', '322', '2', 'credit_card', '2017-03-26 23:36:40'],
 ['12', '40', '983', '322', '2', 'debit', '2017-03-12 17:58:30'],
 ['13', '54', '799', '266', '2', 'credit_card', '2017-03-16 14:15:34'],
 ['14', '100

In [23]:
# Now to put it into a dataframe
import pandas as pd
column_names = data[0]

# I have set the index and order_id to be the same
df = pd.DataFrame(data[1:], columns=column_names, index=[row[0] for row in data[1:]])
df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
1,1,53,746,224,2,cash,2017-03-13 12:36:56
2,2,92,925,90,1,cash,2017-03-03 17:38:52
3,3,44,861,144,1,cash,2017-03-14 4:23:56
4,4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4996,4996,73,993,330,2,debit,2017-03-30 13:47:17
4997,4997,48,789,234,2,cash,2017-03-16 20:36:16
4998,4998,56,867,351,3,cash,2017-03-19 5:42:42
4999,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [32]:
# First instinct is to try and recreate that $3145.13 amount
# Cast as int
df[['order_amount', 'total_items']] = df[['order_amount', 'total_items']].apply(pd.to_numeric)
df.dtypes
df.describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


a. I can see here that the quartiles all look very reasonable, but there is an enormous outlier that should be considered when looking for the average. 
 
 b. Instead, the median would be a much better suited metric for the middle of the data. The median is not influenced by outliers and resists skewness in that regard.   

c. We can see from above that the **median**, or 50th percentile is **$284**

## Question 2: For this question you’ll need to use SQL. 
Follow this [link](https://www.w3schools.com/SQL/TRYSQL.ASP?FILENAME=TRYSQL_SELECT_ALL) to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

*How many orders were shipped by Speedy Express in total?*

```
SELECT COUNT(ORDERID)
FROM [Orders]
WHERE SHIPPERID = 1
```
RESULT: `54`  

 
*What is the last name of the employee with the most orders?*

```
SELECT LASTNAME FROM [Employees]
WHERE EMPLOYEEID = (
SELECT EMPLOYEEID FROM [Orders]
GROUP BY EMPLOYEEID
ORDER BY COUNT(ORDERID) DESC
LIMIT 1
)
```
LastName: 
`Peacock`  
Here the where condition is answered by another query   


*What product was ordered the most by customers in Germany?*


```
SELECT PRODUCTNAME FROM PRODUCTS
WHERE PRODUCTID = (
SELECT PRODUCTID FROM OrderDetails AS OD
JOIN ORDERS AS O ON O.ORDERID = OD.ORDERID
WHERE CUSTOMERID IN (
SELECT CUSTOMERID FROM [Customers]
WHERE COUNTRY = 'Germany'
)
GROUP BY PRODUCTID
ORDER BY SUM(QUANTITY) DESC LIMIT 1
)
```

ProductName: `Boston Crab Meat`

In this query I first (in order of my personal operations, not the query itself) find which customerids are from Germany, I think look at a Join of Order and OrderDetails to access order amounts and customer ids. Now I can calculate the sum(quantities) for the group by ProductID for customers from Germany. Finally we can lookup what that product is called.
